Machine Learning
Para trabajar con sistemas de recomendación, emplearemos la librería Surprise de la que dejamos aquí la documentación.

Tendremos que llevar nuestro dataset al formato con el que trabaja esta librería.

In [ ]:
# Instalamos surprise

pip install scikit-surprise

In [ ]:
# Importamos librerias

import pandas as pd
import sys
import surprise
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
     

In [ ]:

reader = Reader()

In [ ]:
data_score = pd.read_csv('/content/drive/MyDrive/Formación/Henry/PI N°1 - MLOps/score.csv', sep=',', encoding='latin-1')
data_movies = pd.read_csv('/content/drive/MyDrive/Formación/Henry/PI N°1 - MLOps/plataformas_prom.csv', sep=',', encoding='latin-1')

In [ ]:
data_score = data_score[ ['userId', 'rating', 'movieId'] ]
data_score.head(3)

In [ ]:

data_movies = data_movies[ ['Id', 'title'] ]
data_movies.head(3)

In [ ]:
# Unimos ambos dataset, para obtener el nombre de las peliculas con sus votaciones

data_movies_score = pd.merge(left=data_score, right=data_movies, how='left', left_on='movieId', right_on='Id')
data_movies_score.head()

In [ ]:

N_filas = 100000 # Limitamos el dataset a N_filas

data = Dataset.load_from_df( data_movies_score[ ['userId', 'movieId', 'rating']][:N_filas], reader)

In [ ]:

# Separamos nuestros datos

trainset, testset = train_test_split (data, test_size=.25)

In [ ]:
# Usaremos un modelo de Singular Value Decomposition

from surprise import SVD
model = SVD()

In [ ]:
# Entrenamos el modelo

model.fit(trainset)

In [ ]:
# Predecimos

predictions = model.test(testset)

In [ ]:
predictions[1]

In [ ]:

# Hacemos una predicción al azar para usuario y película

model.predict(1, 'as680')

In [ ]:
# Tomaremos un usuario para hacerle una recomendación

usuario = 682
rating = 4   # Tomamos películas a las que haya calificado con 4 o 5 estrellas
df_user = data_movies_score[(data_score['userId'] == usuario) & (data_score['rating'] >= rating)]
df_user = df_user.reset_index(drop=True)
# df_user['Name'] = data_movies['title'].loc[df_user.userId].values
df_user

In [ ]:
recomendaciones_usuario = data_movies.iloc[:4499].copy()
print(recomendaciones_usuario.shape)
recomendaciones_usuario.head()

In [ ]:
# Debemos extraer las películas que ya ha visto

usuario_vistas = data_score[ data_score['userId'] == usuario]
print(usuario_vistas.shape)
usuario_vistas.head()

In [ ]:
# Recomendamos

recomendaciones_usuario['Estimate_Score'] = recomendaciones_usuario['Id'].apply(lambda x: model.predict(usuario, x).est)

In [ ]:
recomendaciones_usuario = recomendaciones_usuario.sort_values('Estimate_Score', ascending=False)
recomendaciones_usuario.head()

In [ ]:
# Evaluación

from surprise import accuracy

accuracy.rmse(predictions)